# 맛집 지도에 표시
맛집 칼럼에서 맛집 이름, 전화번호, 주소를 받아 지도에 표시하는 코드입니다.

# module import
## 사용 모듈 : 
### requests
웹 request, response를 처리하기 위한 모듈
### BeautifulSoup
request의 html 데이터를 DOM Object처럼 처리하기 위한 모듈
### folium
좌표 데이터를 지도로 시각화하기 위한 모듈
### re
정규표현식 사용을 위한 모듈

In [19]:
from bs4 import BeautifulSoup as bs
import requests
import folium
import re

# 함수 작성
## 공통
### get_avg_point
#### parameter
point_list : list(sequence) 자료형으로 x,y값을 담아 전달
#### return
y, x (확장성을 위하여 위도(y), 경도(x)값으로 뒤집어서 반환)
## folium 사용
### getLatLng
#### parameter
addr : str 자료형으로 주소값을 전달
#### return
##### 좌표값 수신 성공
x, y로 좌표값을 반환
##### 좌표값 수신 실패
주소를 공백단위로 끊어 더 넓은 범위로 다시 호출
아예 부르지 못했을 경우 0, 0 좌표로 반환
### add_marker
#### parameter
mapobj : marker를 그릴 Map(folium) 오브젝트를 전달
x : x좌표(경도) 전달
y : y좌표(위도) 전달
info : 맛집 정보를 전달(info[0] : 가게이름, info[1] : 전화번호)
#### return
없음 (이미 Map object에 그렸으므로)

In [20]:
def getLatLng(addr):
#     addr = '서울특별시 구로구 디지털로 306'
    app_key = 'KakaoAK' + ' 78106cafe4d656639f171b177dda611f'
    url = 'https://dapi.kakao.com/v2/local/search/address.json'
    params = {
        'query': addr,
        'page': '',
        'AddressSize': ''
    }
    headers  = {
        'Authorization': app_key
    }
    resp = requests.get(url, params=params, headers=headers)
    if resp.json()['documents'] != []:
        json = resp.json()['documents'][0]
        return (json['x'], json['y'])
    else:
        fixed_addr = addr.replace(addr.split(' ')[-1], '').strip()
        if fixed_addr == '':
            print('불러오지 못했습니다 : ', addr)
            return (0, 0)
        else:
            return getLatLng(fixed_addr)

def add_marker(mapobj, x, y, info):
    folium.Marker((y,x), popup=(info[0] + '\n' + info[1]), icon=folium.Icon(color='red', icon='info-sign')).add_to(mapobj)

def get_avg_point(point_list):
    x = sum(map(lambda i:float(i[0]), point_list)) / len(point_list)
    y = sum(map(lambda i:float(i[1]), point_list)) / len(point_list)
    return y,x

# 칼럼에서 맛집 정보 가공
## 필요한 정보
### 대상 url
url = 'https://www.wikitree.co.kr/articles/217101'
### 공백처리
response 객체의 &nbsp; 문자열을 공백 ' '으로 치환
resp.text.replace('&nbsp;', ' ')
### 정규표현식 패턴 
전화번호 : '^\d{2,3}-\d{3,4}-\d{4}'  
주소 : '^[가-힣 ]{0,4}시?[가-힣]{1,3}구 [가-힣]{0,3}동?[0-9 ]'
## 결과
### 맛집 정보 리스트
리스트 자료형, contents : tuple(가게이름, 전화번호, 주소)
### 맛집 좌표 리스트
리스트 자료형, contents : tuple(x좌표 , y좌표)

In [21]:
# load html
url = 'https://www.wikitree.co.kr/articles/217101'
resp = requests.get(url)

# dataset
soup = bs(resp.text.replace('&nbsp;', ' '),'html.parser')
news_content = soup.select_one('div#wikicon')
store_list = []
tel_list = []
addr_list = []
stores = list(map(lambda i : i.text.split('회 '), news_content.select('strong')))
tel_p = re.compile('^\d{2,3}-\d{3,4}-\d{4}')
addr_p = re.compile('^[가-힣 ]{0,4}시?[가-힣]{1,3}구 [가-힣]{0,3}동?[0-9 ]')
# addr_err = re.compile('^동[0-9 ]{0,3}-?[0-9]{0,2}')

# create each list
for store in stores:
    if len(store) > 1:
        store_list.append(store[-1])
tel_list = news_content.find_all(string = tel_p)
addr_list = news_content.find_all(string = addr_p)
store_total = [(store.strip(), tel.strip(), addr.strip()) for store,tel,addr in zip(store_list, tel_list, addr_list)]

# test 
addr_point_list = list(map(lambda i: getLatLng(i[2]),store_total))

# 가공한 데이터를 지도로 시각화(5개만!)
## 필요한 정보
맛집 정보 리스트
맛집 좌표 리스트
## 결과
시각화된 지도

In [22]:
import random

In [23]:
random_five = []
while True: 
    rand_one = random.randint(1,20)
    if rand_one not in random_five:
        random_five.append(rand_one)
    if len(random_five) >= 5:
        break
random_five.sort()
random_five

[5, 6, 7, 10, 17]

In [30]:
addr_point_list_five = []
store_total_five = []
for i in random_five:
    addr_point_list_five.append(addr_point_list[i])
    store_total_five.append(store_total[i])
addr_point_list_five

[('127.00839172712', '37.5900669779109'),
 ('127.027897674019', '37.4986090348489'),
 ('126.955664169252', '37.5551587300272'),
 ('127.019786406742', '37.5210732573275'),
 ('126.923556958229', '37.5620838593197')]

In [34]:
map_food = folium.Map(location = get_avg_point(addr_point_list_five), zoom_start=11)
cnt = 0
for x,y in addr_point_list_five:
    add_marker(map_food, x, y, store_total_five[cnt])
    cnt += 1
map_food.save('map.html')

In [35]:
map_food

# 개선사항
marker의 범위에 따른 zoom 범위 조정  
folium이 현재 marker에 한글을 지원하지 못함(20.06.11)